<a href="https://colab.research.google.com/github/naman065/Inter-IIT-PrepCamp/blob/main/Faster_RCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FASTER R-CNN

In [1]:
# Install Detectron2
!pip install torch torchvision torchaudio
!pip install 'git+https://github.com/facebookresearch/detectron2.git'


  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-6n_l7z2b
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-6n_l7z2b
  Resolved https://github.com/facebookresearch/detectron2.git to commit 5b72c27ae39f99db75d43f18fd1312e1ea934e60
  Preparing metadata (setup.py) ... done


In [2]:
# Download and extract Pascal VOC 2007 dataset
!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar
!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtest_06-Nov-2007.tar
!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCdevkit_08-Jun-2007.tar

!tar -xvf VOCtrainval_06-Nov-2007.tar
!tar -xvf VOCtest_06-Nov-2007.tar
!tar -xvf VOCdevkit_08-Jun-2007.tar


Streaming output truncated to the last 5000 lines.
VOCdevkit/VOC2007/JPEGImages/000817.jpg
VOCdevkit/VOC2007/JPEGImages/000819.jpg
VOCdevkit/VOC2007/JPEGImages/000821.jpg
VOCdevkit/VOC2007/JPEGImages/000824.jpg
VOCdevkit/VOC2007/JPEGImages/000825.jpg
VOCdevkit/VOC2007/JPEGImages/000833.jpg
VOCdevkit/VOC2007/JPEGImages/000835.jpg
VOCdevkit/VOC2007/JPEGImages/000836.jpg
VOCdevkit/VOC2007/JPEGImages/000837.jpg
VOCdevkit/VOC2007/JPEGImages/000838.jpg
VOCdevkit/VOC2007/JPEGImages/000839.jpg
VOCdevkit/VOC2007/JPEGImages/000840.jpg
VOCdevkit/VOC2007/JPEGImages/000841.jpg
VOCdevkit/VOC2007/JPEGImages/000844.jpg
VOCdevkit/VOC2007/JPEGImages/000846.jpg
VOCdevkit/VOC2007/JPEGImages/000852.jpg
VOCdevkit/VOC2007/JPEGImages/000853.jpg
VOCdevkit/VOC2007/JPEGImages/000856.jpg
VOCdevkit/VOC2007/JPEGImages/000858.jpg
VOCdevkit/VOC2007/JPEGImages/000861.jpg
VOCdevkit/VOC2007/JPEGImages/000864.jpg
VOCdevkit/VOC2007/JPEGImages/000866.jpg
VOCdevkit/VOC2007/JPEGImages/000869.jpg
VOCdevkit/VOC2007/JPEGImages/

In [10]:
from detectron2.data import DatasetCatalog, MetadataCatalog

# Unregister the dataset if it was already registered
DatasetCatalog.remove("voc_2007_trainval")
DatasetCatalog.remove("voc_2007_test")
MetadataCatalog.remove("voc_2007_trainval")
MetadataCatalog.remove("voc_2007_test")

# Register the dataset again
from detectron2.data.datasets import register_pascal_voc

register_pascal_voc('voc_2007_trainval', '/content/VOCdevkit/VOC2007', 'trainval', '2007')
register_pascal_voc('voc_2007_test', '/content/VOCdevkit/VOC2007', 'test', '2007')


In [11]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2 import model_zoo

# Create config
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))

# Set dataset
cfg.DATASETS.TRAIN = ("voc_2007_trainval",)
cfg.DATASETS.TEST = ("voc_2007_test",)
cfg.DATALOADER.NUM_WORKERS = 2

# Set output directory
cfg.OUTPUT_DIR = './output'

# Set other training parameters
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.001  # pick a good learning rate
cfg.SOLVER.MAX_ITER = 5000    # number of iterations
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 20  # Pascal VOC has 20 classes

# Create trainer and start training
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()


[08/24 05:09:37 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

model_final_280758.pkl: 167MB [00:05, 32.6MB/s]                           
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}


[08/24 05:09:43 d2.engine.train_loop]: Starting training from iteration 0


/usr/local/lib/python3.10/dist-packages/torch/functional.py:512: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3587.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[08/24 05:09:56 d2.utils.events]:  eta: 0:35:37  iter: 19  total_loss: 3.869  loss_cls: 2.991  loss_box_reg: 0.8721  loss_rpn_cls: 0.0107  loss_rpn_loc: 0.01257    time: 0.4150  last_time: 0.4723  data_time: 0.0184  last_data_time: 0.0138   lr: 1.9981e-05  max_mem: 3034M
[08/24 05:10:12 d2.utils.events]:  eta: 0:35:29  iter: 39  total_loss: 3.54  loss_cls: 2.713  loss_box_reg: 0.8418  loss_rpn_cls: 0.01306  loss_rpn_loc: 0.01262    time: 0.4138  last_time: 0.4098  data_time: 0.0123  last_data_time: 0.0137   lr: 3.9961e-05  max_mem: 3034M
[08/24 05:10:20 d2.utils.events]:  eta: 0:34:11  iter: 59  total_loss: 2.811  loss_cls: 1.882  loss_box_reg: 0.8631  loss_rpn_cls: 0.01114  loss_rpn_loc: 0.02131    time: 0.4044  last_time: 0.4148  data_time: 0.0117  last_data_time: 0.0196   lr: 5.9941e-05  max_mem: 3034M
[08/24 05:10:29 d2.utils.events]:  eta: 0:34:09  iter: 79  total_loss: 2.024  loss_cls: 1.195  loss_box_reg: 0.7774  loss_rpn_cls: 0.003571  loss_rpn_loc: 0.01771    time: 0.4071  las

In [12]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

# Create evaluator and run evaluation
evaluator = COCOEvaluator("voc_2007_test", output_dir="./output")
val_loader = build_detection_test_loader(cfg, "voc_2007_test")
inference_on_dataset(trainer.model, val_loader, evaluator)


[08/24 05:46:17 d2.evaluation.coco_evaluation]: Trying to convert 'voc_2007_test' to COCO format ...
[08/24 05:46:17 d2.data.datasets.coco]: Converting annotations of dataset 'voc_2007_test' to COCO format ...)
[08/24 05:46:17 d2.data.datasets.coco]: Converting dataset dicts into COCO format
[08/24 05:46:20 d2.data.datasets.coco]: Conversion finished, #images: 4952, #annotations: 14976
[08/24 05:46:20 d2.data.datasets.coco]: Caching COCO format annotations at './output/voc_2007_test_coco_format.json' ...
[08/24 05:46:21 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[08/24 05:46:21 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
[08/24 05:46:21 d2.data.common]: Serializing 4952 elements to byte tensors and concatenating them all ...
[08/24 05:46:21 d2.data.common]: Serialized dataset takes 2.27 MiB
[08/24 05:46:21 d2.e

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


[08/24 05:46:22 d2.evaluation.evaluator]: Inference done 11/4952. Dataloading: 0.0072 s/iter. Inference: 0.1264 s/iter. Eval: 0.0003 s/iter. Total: 0.1339 s/iter. ETA=0:11:01
[08/24 05:46:27 d2.evaluation.evaluator]: Inference done 55/4952. Dataloading: 0.0046 s/iter. Inference: 0.1112 s/iter. Eval: 0.0003 s/iter. Total: 0.1163 s/iter. ETA=0:09:29
[08/24 05:46:32 d2.evaluation.evaluator]: Inference done 102/4952. Dataloading: 0.0033 s/iter. Inference: 0.1079 s/iter. Eval: 0.0003 s/iter. Total: 0.1116 s/iter. ETA=0:09:01
[08/24 05:46:38 d2.evaluation.evaluator]: Inference done 148/4952. Dataloading: 0.0031 s/iter. Inference: 0.1078 s/iter. Eval: 0.0003 s/iter. Total: 0.1114 s/iter. ETA=0:08:55
[08/24 05:46:43 d2.evaluation.evaluator]: Inference done 193/4952. Dataloading: 0.0036 s/iter. Inference: 0.1078 s/iter. Eval: 0.0003 s/iter. Total: 0.1118 s/iter. ETA=0:08:52
[08/24 05:46:48 d2.evaluation.evaluator]: Inference done 239/4952. Dataloading: 0.0033 s/iter. Inference: 0.1078 s/iter. E

OrderedDict([('bbox',
              {'AP': 46.07694530890001,
               'AP50': 75.76240206058338,
               'AP75': 49.994555042808294,
               'APs': 21.98398048066576,
               'APm': 38.34272796282164,
               'APl': 52.35255076261686,
               'AP-aeroplane': 53.63388753010211,
               'AP-bicycle': 53.7869134869336,
               'AP-bird': 41.67937004192447,
               'AP-boat': 28.70148005774445,
               'AP-bottle': 38.45547559490244,
               'AP-bus': 56.8583262215451,
               'AP-car': 61.21842320840864,
               'AP-cat': 45.711697250104336,
               'AP-chair': 33.170356232085986,
               'AP-cow': 53.46228872959043,
               'AP-diningtable': 38.65014556327123,
               'AP-dog': 41.38966078320523,
               'AP-horse': 56.741901845573885,
               'AP-motorbike': 48.67636559340433,
               'AP-person': 53.73884097359273,
               'AP-pottedplant': 